In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# google sheet: https://docs.google.com/spreadsheets/d/14L32EfCmr2asv85i08fla6jf1KakJVcLYaJMkXQ4_p0/edit#gid=0    

#Note: Data has been filtered/changed a bit from orginal form to demonstrate null and outlier handling.


sheet_url = 'https://docs.google.com/spreadsheets/d/14L32EfCmr2asv85i08fla6jf1KakJVcLYaJMkXQ4_p0/edit#gid=0'    

csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

df = pd.read_csv(csv_export_url)

In [ ]:
df.head()

In [ ]:
df.hist()
plt.tight_layout();

In [ ]:
df.info()



### How to handle nulls and outliers

### How do I handle missing data (nulls) in my dataset?

- Drop the nulls
    - drop columns
    - drop rows
- Impute missing values
    - fillna with mean, median, mode of the column
    - Impute using some other algorithms - KNN imputer
- Use algorithms which can handle nulls
    - Ignores missing values
- Boolean flags for missing data 
    - (https://scikit-learn.org/stable/modules/generated/sklearn.impute.MissingIndicator.html#sklearn.impute.MissingIndicator)


In [ ]:
# How many nulls have in each column?

df.isnull().sum()

In [ ]:
# % values missing in each column

df.isnull().sum()/len(df)*100

In [ ]:
# How many nulls have in each row?

df.isnull().sum(axis =1).value_counts()

In [ ]:
# Drop columns using dropna() based on a threshold

# threshold: Require that many non-NA values to survive. 

df.dropna(axis = 1, thresh = 0.5 * len(df.index))

In [ ]:
# Drop rows using dropna() based on a threshold

# threshold: only drop when # of nulls in a row is above the threshold value

df.dropna(axis = 0, thresh = 0.5 * len(df.columns))

In [ ]:
def handle_missing_values(df, prop_required_row = 0.5, prop_required_col = 0.5):
    ''' function which takes in a dataframe, required notnull proportions of non-null rows and columns.
    drop the columns and rows columns based on theshold:'''
    
    #drop columns with nulls
    threshold = int(prop_required_col * len(df.index)) # Require that many non-NA values.
    df.dropna(axis = 1, thresh = threshold, inplace = True)
    
    #drop rows with nulls
    threshold = int(prop_required_row * len(df.columns)) # Require that many non-NA values.
    df.dropna(axis = 0, thresh = threshold, inplace = True)
    
    
    return df

In [ ]:
# use the function on our dataset
df = handle_missing_values(df, prop_required_row = 0.5, prop_required_col = 0.5)
df.head()

In [ ]:
df.info()

In [ ]:
# check nulls again
df.isnull().sum()

In [ ]:
# drop null rows for specific columns only

df = df[df.YearsCode.notnull()]
df = df[df.YearsCodePro.notnull()]
df = df[df.Age1stCode.notnull()]

df.isnull().sum()

In [ ]:
# what is the most common/frequent observation in Gender in train dataset?
df.Gender.value_counts()/len(df)*100

In [ ]:
# Fill NaNs with mode

df.Gender.mode()[0]

df['Gender'] = df.Gender.fillna(df.Gender.mode()[0])

In [ ]:
# check nulls again
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# can I simply convert the object columns to float/int using 'astype'?
df['Age1stCode'] = df.Age1stCode.astype('int64')

In [ ]:
# We have some strings these columns preventing them to be converted to int dtypes
df.YearsCode.value_counts()

In [ ]:
df.Age1stCode.value_counts()

In [ ]:
# use replace function to reaplce strings with values

df.replace('Younger than 5 years',4, inplace = True )
df.replace('Older than 85', 85, inplace = True )

df.replace('More than 50 years', 50, inplace = True )

df.replace('Less than 1 year', 0, inplace = True )

In [ ]:
# Now we can change datatype for these columns from 'object' to 'int64'

df['Age1stCode'] = df.Age1stCode.astype('int64')
df['YearsCode'] = df.YearsCode.astype('int64')
df['YearsCodePro'] = df.YearsCodePro.astype('int64')

In [ ]:
df.info()

I will try to impute age using other columns, but I need to split my data first

In [ ]:
# split the data in train, validate and test dataframes
train, test = train_test_split(df,test_size=0.2, random_state=42)
train, validate = train_test_split(train,test_size=0.25, random_state=42)

In [ ]:
# check the shape of three dataframes
train.shape,validate.shape, test.shape

In [ ]:
# use KNN imputer to find missing values for 'Age' 
from sklearn.impute import KNNImputer

#Use numeric columns that you want to use for imputation
X_numeric = train[['Age', 'Age1stCode', 'YearsCode', 'YearsCodePro']]

# define the thing
imputer = KNNImputer(n_neighbors=1)

# fit the thing (or fit and use with fit_transform) only on train!
train_imputed = imputer.fit_transform(X_numeric)
train_imputed

In [ ]:
# check there is no null in imputed columns
pd.DataFrame(train_imputed).isnull().sum()

In [ ]:
# convert imputed array to a dataframe
train_imputed = pd.DataFrame(train_imputed, index = train.index)
train_imputed.head()

In [ ]:
# assign imputed values to the age column
train['Age'] = train_imputed[[0]]

In [ ]:
train.info()

In [ ]:
# do the transformation on validate and test
validate_imputed = imputer.transform(validate[['Age', 'Age1stCode', 'YearsCode', 'YearsCodePro']])
test_imputed = imputer.transform(test[['Age', 'Age1stCode', 'YearsCode', 'YearsCodePro']])

In [ ]:
# convert arrarys from above cell in dataframes
validate_imputed = pd.DataFrame(validate_imputed, index = validate.index)
test_imputed = pd.DataFrame(test_imputed, index = test.index)


In [ ]:
# assign imputed to 'Age' column for validate and test dataframes
validate['Age'] = validate_imputed[[0]]
test['Age'] = test_imputed[[0]]

### Outliers
- Data point(s) that differs significantly from other observations
- Could be due to chance, measurement errors, transcription error, sampling error/bias,

Question to ask:
- Is the outlier part of population that I want to explore/model?

In [ ]:
# plot histograms from Age and Compensation

plt.figure(figsize = (12,6))

plt.subplot(121)
df.Age.hist(bins = 100)
plt.title('Age')

plt.subplot(122)
df.Comp.hist(bins = 100)
plt.title('Compensation')

In [ ]:
# Boxplot for Age and Compensation

plt.figure(figsize = (12,6))

plt.subplot(121)
sns.boxplot(y = 'Age', data = df, whis = 3)
plt.title('Age')

plt.subplot(122)
plt.title('Compensation')
sns.boxplot(y = 'Comp', data = df, whis = 3)


### Handling Outliers:
- Use domain knowledge/business rules to have cut-offs
- Use IQR method to exclude outliers
- Cap/Trim max value 
- transformation
- Impute new value (if you know outlier is a mistake)

In [ ]:
# Calculate q1, q3 and iqr for Age

q1 = df.Age.quantile(0.25)
q3 = df.Age.quantile(0.75)
iqr = q3 - q1
k = 3

# calculate upper and lower ranges
upper_bound_Age =  q3 + k * iqr
lower_bound_Age =  q1 - k * iqr

In [ ]:
# Calculate q1, q3 and iqr for Comp


q1 = df.Comp.quantile(0.25)
q3 = df.Comp.quantile(0.75)
iqr = q3 - q1
k = 3

# calculate upper and lower ranges
upper_bound_Comp = q3 + k * iqr
lower_bound_Comp = q1 - k * iqr

In [ ]:
# Filter dataframe using multiple conditions

df[(df.Age < upper_bound_Age) & (df.Comp < upper_bound_Comp)]

In [ ]:
# Cap the limit

# df['column'] = np.where(this_is_true(?), do_this, else_do_that)

df.Comp_capped = np.where(df.Comp > 1_000_000, df.Comp == 1_000_000, df.Comp)